In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../../../tools'))
import numpy as np
from verifyoutput import *
from lwnn2torch import *
from tf2lwnn import *
import onnx
import onnxruntime
%matplotlib inline

In [ ]:
DEMO_MODEL='Basic_LSTM_S.pb'
DEMO_INPUT='/mnt/d/tmp/speech_dataset/yes/0a9f9af7_nohash_1.wav'
LABELS=['_silence_','_unknown_','yes','no','up','down','left','right','on','off','stop','go']

In [ ]:
wav_data = np.fromfile(DEMO_INPUT, np.int8).reshape(1, -1)
converter = TfConverter(DEMO_MODEL, 'KWS', {'wav_data': wav_data})
lwnn_model = converter.convert()

In [ ]:
for layer in lwnn_model:
    print(layer)

In [ ]:
itensor = converter.tensors['wav_data']
otensor = converter.tensors['labels_softmax']
lstm_i = converter.tensors['LSTM-Layer/lstm/transpose']
lstm_o = converter.tensors['LSTM-Layer/lstm/rnn/while/Exit_3']

In [ ]:
result, lstm_i_result, lstm_o_result = converter.sess.run([otensor, lstm_i, lstm_o], {itensor: wav_data.tobytes() })

In [ ]:
for k,p in zip(LABELS, result.tolist()[0]):
    print('%s: %.2f'%(k, p))

In [ ]:
lstm_i_result.shape, lstm_o_result.shape

In [ ]:
for node in converter.onnx_model.graph.node:
    if(node.op_type == 'LSTM'):
        lstm_node = node
    if(node.op_type == 'Squeeze'):
        sq_node = node
lstm_node, sq_node

In [ ]:
x = onnx.helper.make_tensor_value_info(lstm_node.input[0], onnx.TensorProto.FLOAT, lstm_i_result.shape)
attrs = {}
for attr in lstm_node.attribute:
    v = onnx.helper.get_attribute_value(attr)
    attrs[attr.name] = v
node = onnx.helper.make_node(
            'LSTM',
            name = lstm_node.name,
            inputs=lstm_node.input[:4],
            outputs=lstm_node.output,
            **attrs)
outputs = [onnx.helper.make_tensor_value_info(o, onnx.TensorProto.FLOAT, None) for o in sq_node.output]
graph = onnx.helper.make_graph(
            nodes = [node, sq_node],
            name = 'LSTM',
            inputs = [x],
            outputs = outputs,
            value_info = [],
            initializer = converter.onnx_model.graph.initializer)
model = onnx.helper.make_model(graph, producer_name='lwnn-nhwc')
onnx.save(model, '.tmp.onnx')

In [ ]:
sess = onnxruntime.InferenceSession('.tmp.onnx')

In [ ]:
rs = sess.run(None, {lstm_node.input[0]: lstm_i_result })

In [ ]:
rs[0].shape

In [ ]:
compare(lstm_o_result, rs[0])